# Using MCP Tools with Azure AI Foundry Agents

This notebook demonstrates how to use Model Context Protocol (MCP) tools with Azure AI Foundry agents. Azure AI Foundry provides seamless integration with hosted MCP servers, eliminating infrastructure management while providing secure, controlled access to external tools.

## What You'll Learn:
- Basic hosted MCP tool integration with Azure AI Foundry agents
- Multi-tool MCP configuration with different approval modes
- Azure AI observability for monitoring and tracing
- Microsoft Learn MCP server integration for documentation queries

## Key Features:
- **Hosted MCP Server**: Managed by Azure AI Foundry, no infrastructure to maintain
- **Persistent Agents**: Server-side agent creation with stateful conversations
- **Tool Approval Workflow**: Configurable approval mechanisms for MCP tool invocations
- **Observability**: Built-in monitoring and tracing for production scenarios

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for Azure AI agent functionality with MCP integration.

In [24]:
import os
import asyncio
from pathlib import Path
from dotenv import load_dotenv

from agent_framework import HostedMCPTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify required environment variables
print("Checking environment variables...")
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

if endpoint:
    print(f"✅ AZURE_AI_PROJECT_ENDPOINT: {endpoint[:50]}...")
    print(f"✅ AZURE_AI_MODEL_DEPLOYMENT_NAME: {model}")
else:
    print("❌ AZURE_AI_PROJECT_ENDPOINT is not set!")
    print("Please configure your .env file with the required variables.")

Checking environment variables...
✅ AZURE_AI_PROJECT_ENDPOINT: https://agentic-ai-labs.services.ai.azure.com/api/...
✅ AZURE_AI_MODEL_DEPLOYMENT_NAME: gpt-4o


## Example 1: Basic MCP Integration

This example demonstrates the simplest way to create an Azure AI Foundry agent with a hosted MCP tool. The agent can search Microsoft Learn documentation to answer questions.

In [25]:
async def basic_foundry_mcp_example():
    """Basic example of Azure AI Foundry agent with hosted MCP tools."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        # Enable Azure AI observability (optional but recommended)
        #print("🔍 Setting up Azure AI observability...")
        #await chat_client.setup_azure_ai_observability()
        print("✅ Azure AI observability enabled\n")

        # Create agent with hosted MCP tool
        print("🤖 Creating MicrosoftLearnAgent with hosted MCP tool...")
        agent = chat_client.create_agent(
            name="MicrosoftLearnAgent", 
            instructions="You answer questions by searching Microsoft Learn content only.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
                approval_mode="never_require",  # Auto-approve tool calls
            ),
        )
        print(f"✅ Created agent: {agent.name}\n")

        # Simple query without approval workflow
        query = "Please summarize the Azure AI Agent documentation related to MCP tool calling?"
        print(f"🤔 Query: {query}\n")
        
        result = await agent.run(query)
        
        print(f"🤖 Response:\n{result.text}")
        print(f"\n✅ Basic example completed!")

# Run the basic example (use await in notebooks, asyncio.run() in scripts)
await basic_foundry_mcp_example()

✅ Azure AI observability enabled

🤖 Creating MicrosoftLearnAgent with hosted MCP tool...
✅ Created agent: MicrosoftLearnAgent

🤔 Query: Please summarize the Azure AI Agent documentation related to MCP tool calling?

🤖 Response:
Azure AI Agent's Model Context Protocol (MCP) integration allows AI agents to dynamically leverage external tools and services for enhanced capabilities. Below is a summary of key documentation points about MCP tool-calling:

1. **Enabling MCP Tools in Agents:**
   - Azure AI agents can use MCP tools by connecting to MCP servers (custom, self-hosted, or Databricks-managed).
   - MCP servers provide tools for real-time data, APIs, or service access.
   - To initialize an MCP-enabled agent, provide server configuration details (e.g., endpoint URL, tool labels, authentication).

2. **Agent Integration with MCP:**
   - Tools can be added to agents via Azure AI Foundry or Visual Studio Code extensions.
   - Configuration involves registering MCP tools in the agent's 

## Example 2: Multi-Tool MCP Configuration

This example demonstrates using multiple hosted MCP tools with different approval modes. This is useful when you want to integrate multiple external services with varying security requirements.

In [ ]:
async def multi_tool_mcp_example():
    """Example using multiple hosted MCP tools with different approval modes."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        #print("🔍 Setting up Azure AI observability...")
        #await chat_client.setup_azure_ai_observability()
        print("✅ Azure AI observability enabled\n")

        # Create agent with multiple MCP tools
        print("🤖 Creating MultiToolAgent with multiple MCP tools...")
        agent = chat_client.create_agent(
            name="MultiToolAgent",
            instructions="You can search documentation and provide comprehensive answers.",
            tools=[
                HostedMCPTool(
                    name="Microsoft Learn MCP",
                    url="https://learn.microsoft.com/api/mcp",
                    approval_mode="never_require",  # Auto-approve documentation searches
                ),
                # Note: Add more MCP tools here as needed
                #Example with custom headers:
                HostedMCPTool(
                    name="GitHub MCP", 
                    url="https://api.github.com/mcp",
                    approval_mode="always_require",  # Require approval for GitHub operations
                    headers={"Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}"},
                ),
            ],
        )
        print(f"✅ Created agent: {agent.name}\n")

        # Query that will use the MCP tools
        query = "What are the key features of Azure AI Foundry?"
        print(f"🤔 Query: {query}\n")
        
        result = await agent.run(query)
        
        print(f"🤖 Response:\n{result.text}")
        print(f"\n✅ Multi-tool example completed!")

# Run the multi-tool example
await multi_tool_mcp_example()

✅ Azure AI observability enabled

🤖 Creating MultiToolAgent with multiple MCP tools...
✅ Created agent: MultiToolAgent

🤔 Query: What are the key features of Azure AI Foundry?



[2025-12-01 15:39:24 - /Users/anuragkaruparti/agentic-ai-lab/.venv/lib/python3.12/site-packages/agent_framework_azure_ai/_chat_client.py:676 - ERROR] Error processing stream: MCP Connector error. Http status: 424, error details: Error retrieving tool list from MCP server: 'GitHub_MCP'. Http status code: 404 (Not Found)


ServiceResponseException: MCP Connector error. Http status: 424, error details: Error retrieving tool list from MCP server: 'GitHub_MCP'. Http status code: 404 (Not Found)

## Example 3: Thread-Based Conversation with MCP Tools

This example demonstrates how to use threads to maintain conversation context across multiple queries while using hosted MCP tools.

In [23]:
async def thread_based_mcp_example():
    """Example showing thread-based conversation with hosted MCP tools."""
    import asyncio
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        print("🤖 Creating DocsAgent with Microsoft Learn MCP...")
        agent = chat_client.create_agent(
            name="DocsAgent",
            instructions="You are a helpful assistant that can help with Microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
                approval_mode="never_require",
            ),
        )
        print(f"✅ Created agent: {agent.name}\n")
        
        thread = agent.get_new_thread()
        print(f"📝 Created new conversation thread\n")
        
        # First query
        query1 = "How to create an Azure storage account using az cli?"
        print(f"=== Query 1 ===")
        print(f"🤔 User: {query1}\n")
        result1 = await agent.run(query1, thread=thread, store=True)
        print(f"🤖 {agent.name}: {result1.text}")
        
        print("\n" + "="*60 + "\n")
        
        # Add delay before second query
        print("⏳ Waiting before next query...")
        await asyncio.sleep(2)
        
        # Second query
        query2 = "What is Foundry"
        print(f"=== Query 2 ===")
        print(f"🤔 User: {query2}\n")
        try:
            result2 = await agent.run(query2, thread=thread, store=True)
            print(f"🤖 {agent.name}: {result2.text}")
        except Exception as e:
            print(f"⚠️ Second query failed: {e}")
            print("💡 This can happen with MCP tools - try running again")
        
        print(f"\n✅ Thread-based conversation completed!")

await thread_based_mcp_example()

🤖 Creating DocsAgent with Microsoft Learn MCP...
✅ Created agent: DocsAgent

📝 Created new conversation thread

=== Query 1 ===
🤔 User: How to create an Azure storage account using az cli?

🤖 DocsAgent: To create an Azure storage account using the Azure CLI, you can use the following command format:

```azurecli
az storage account create \
  --name <STORAGE_ACCOUNT_NAME> \
  --resource-group <RESOURCE_GROUP_NAME> \
  --location <LOCATION> \
  --sku <SKU> \
  --kind <KIND>
```

### Example

For a basic storage account in the `westus` region with locally redundant storage (Standard_LRS), you could use:
```azurecli
az storage account create \
  --name MyStorageAccount \
  --resource-group MyResourceGroup \
  --location westus \
  --sku Standard_LRS \
  --kind StorageV2
```

Here:
- `--name`: Specifies the name of the storage account.
- `--resource-group`: The resource group where the storage account will be placed.
- `--location`: Specifies the Azure region (e.g., `westus`, `eastus`).
- `

[2025-12-01 15:20:44 - /Users/anuragkaruparti/agentic-ai-lab/.venv/lib/python3.12/site-packages/agent_framework_azure_ai/_chat_client.py:676 - ERROR] Error processing stream: Sorry, something went wrong.


⚠️ Second query failed: Sorry, something went wrong.
💡 This can happen with MCP tools - try running again

✅ Thread-based conversation completed!


## Key Takeaways

1. **Hosted MCP Tools**: Enable integration with external Model Context Protocol servers
2. **User Approval Workflows**: Provide security by requiring consent for function calls
3. **Thread Management**: Maintain conversation context across multiple queries
4. **Azure AI Observability**: Built-in monitoring and tracing for agent interactions
5. **Microsoft Learn Integration**: Access to comprehensive Microsoft documentation
6. **Error Handling**: Robust error handling for production scenarios

## Best Practices

1. **Security First**: Always implement proper approval workflows for function calls
2. **Observability**: Enable Azure AI observability for monitoring and debugging
3. **Thread Management**: Use threads to maintain conversation context
4. **Error Handling**: Implement comprehensive error handling for reliability
5. **Custom Approvals**: Tailor approval logic to your specific security requirements
6. **Resource Cleanup**: Properly manage agent and thread lifecycles

## Use Cases

- **Documentation Assistance**: AI-powered help with Microsoft technologies
- **Technical Support**: Automated support with human oversight
- **Knowledge Management**: Organizational knowledge base integration
- **Training and Education**: Interactive learning with documentation
- **Code Generation**: Context-aware code examples and templates
- **Compliance**: Secure function execution with approval workflows